In [1]:
"""
This notebook walks though how to use BuildingMotif Templates.
"""
from rdflib import RDF, URIRef

from buildingmotif.building_motif import BuildingMotif
from buildingmotif.dataclasses.template import Template
from buildingmotif.dataclasses.template_library import TemplateLibrary

In [2]:
"""
As always with BuildingMotif, we start by initing BuildingMotif.
"""
building_motif = BuildingMotif(f"sqlite://")
building_motif.session


In [3]:
"""
Templates must always be conneted to a template library.
"""
tl = TemplateLibrary.create("my_tl")
tl

TemplateLibrary(_id=1, _name='my_tl', _bm=<buildingmotif.building_motif.BuildingMotif object at 0x10aa93f70>)

In [4]:
"""
Create a template through the template library. The body is currently empty.

"""
ahu_template = tl.create_template(
    name="ahu",
    head=["name", "zone"],
)

print(ahu_template)
print(ahu_template.head)
print(ahu_template.body.serialize(format="ttl"))

Template(_id=1, _name='ahu', _head=('name', 'zone'), body=<Graph identifier=a46b4575-3561-4faa-9e08-bd4f196b4694 (<class 'rdflib.graph.Graph'>)>, _bm=<buildingmotif.building_motif.BuildingMotif object at 0x10aa93f70>)
('name', 'zone')




In [5]:
"""
Now we can populate the body.
Note that the nodes "name" and "zone" are variables, or undefinied.
They are the template's head, as defined above. This means their values
will be pass to the template when evaluated.
"""
def as_brick(x):
    return URIRef(f"https://brickschema.org/schema/Brick#{x}")

def as_var(x):
    return URIRef(f"https://buildingmotif.org/var#{x}")

ahu_template.body.add((as_var("name"), RDF.type, as_brick("AHU")))
ahu_template.body.add((as_var("name"), as_brick("feeds"), as_var("zone")))


print(ahu_template.body.serialize(format="ttl"))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<https://buildingmotif.org/var#name> a brick:AHU ;
    brick:feeds <https://buildingmotif.org/var#zone> .




In [6]:
"""
Note that the head is immutable.
"""
try:
    ahu_template.head = ["nope"]
except AttributeError as e:
    print(e)

try:
    ahu_template.head.append("nope")
except AttributeError as e:
    print(e)

Cannot modify head
'tuple' object has no attribute 'append'


In [7]:
"""
In addition to heads, templates may also have dependancies on other templates.
Say we added to ahu_template the following tuple, where "sf" refers to the
following supply_fan template.

We must then create a dependancy between the two, and define how the dependancy
head should be evaluated.
"""
# this new triples requires dependancy "sf"
ahu_template.body.add((as_var("name"), as_brick("hasPart"), as_var("sf")))

# here we define the template we will depend on
sf_template = tl.create_template(
    name="supply_fan",
    head=["name"],
)
sf_template.body.add((as_var("name"), RDF.type, as_brick("Supply_Fan")))

# and finally define the dependancy.
ahu_template.add_dependency(sf_template, {"name": "sf"})

print(ahu_template.get_dependencies())

(Dependency(_template_id=2, args={'name': 'sf'}),)


In [8]:
""" 
Note that all heads must be defined to create a dependancy,
and that there can only be one dependency per directional dependant/dependee pair
"""
building_motif.session.commit()

try:
    ahu_template.add_dependency(sf_template, {"not a head": "sf"})
except ValueError as e:
    print(e)
    
try:
    ahu_template.add_dependency(sf_template, {"name": "sf"})
except Exception as e:
    print(e)
    building_motif.session.rollback()

All args in dependee template 2's head must be in args ({'not a head': 'sf'})
(sqlite3.IntegrityError) UNIQUE constraint failed: deps_association_table.dependant_id, deps_association_table.dependee_id
[SQL: INSERT INTO deps_association_table (dependant_id, dependee_id, args) VALUES (?, ?, ?)]
[parameters: (1, 2, '{"name": "sf"}')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)


In [ ]:
"""TO COME: template evaluation"""